In [176]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn import svm
from sklearn import metrics
import pandas as pd
import numpy as np
from pandas import DataFrame 
import os

In [380]:
#===================================== read datas =====================================
user_id,content_type,timestamp,textual=[],[],[],[]
with open('../data/Reputation_prediction_data_train.txt') as f:    
    for line in f:
        data = line.strip().split()
        user_id.append(data[-3])
        content_type.append(data[-2])
        timestamp.append(data[-1])
        textual.append(data[:-3])
df_data = DataFrame({'user_id': user_id})
df_data['user_id']=user_id
df_data['content_type'] = content_type
df_data['textual'] = textual
#===================================== read labels =====================================
user_id, label = [],[]
with open('../data/Reputation_prediction_labels_train.txt') as f:    
    for line in f:
        data = line.strip().split()
        user_id.append(data[0])
        label.append(data[1])
df_test = DataFrame()
df_test['label']=label
df_test['user_id'] = user_id
#===================================== merge 2 dataframe =====================================
df = pd.merge(df_data, df_test, on='user_id',how='left')
#===================================== group data =====================================
df_group = df.groupby(['user_id','content_type','label']).agg({'textual':'sum'}).add_suffix('_comb').reset_index()
df_group['textual_comb']=[' '.join(i) for i in df_group.textual_comb]
df_group.shape  #(6535, 4)

(6535, 4)

In [381]:
df[190:200]

,user_id,content_type,textual,label
190,13,Q,"[basic, premis, restorationist, movement, ear,...",1
191,13,Q,"[distinct, ordin, type, lds, church, recent, l...",1
192,13,Q,"[origin, bridal, veil, christian, wed, practic...",1
193,13,Q,"[accord, kingdom, theolog, man, hold, secular,...",1
194,13,Q,"[differ, elder, led, elder, rule, church, gove...",1
195,13,Q,"[distinct, exhibit, true, furqan, book, true, ...",1
196,13,Q,"[matthew, mark, luke, call, synopt, gospel, go...",1
197,14,Q,"[fast, daniel, cyrus, king, persia, reveal, da...",3
198,14,Q,"[pray, bold, pray, person, achiev, pray, selfl...",3
199,15,Q,"[differ, judg, god, question, god, answer, man...",1


In [229]:
A_df = df_group[df_group['content_type']=='A']
C_df = df_group[df_group['content_type']=='C']
E_df = df_group[df_group['content_type']=='E']
F_df = df_group[df_group['content_type']=='F']
Q_df = df_group[df_group['content_type']=='Q']

In [382]:
# df_group = F_df
a_train, a_test, b_train, b_test = train_test_split(df_group.textual_comb,df_group.label, test_size=0.2)

In [263]:
# df_group['textual'].to_frame()
# agg_df.index=agg_df.index.map(int)
# agg_df.sort_index(inplace=True)
# agg_df['label']= df_test['label'].tolist()
# tf_transformer = TfidfTransformer(use_idf=False).fit(X)
# X_train_tf = tf_transformer.transform(X)

In [430]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,6))),
                     ('clf', MultinomialNB())])
text_clf.fit(a_train, b_train)  
predicted = text_clf.predict(a_test)
np.mean(predicted == b_test)   
# avg = 0.58730158730158732,
# 1=0.50917431192660545, ->0.54 0.001
# 2=0.55045871559633031, ->0.628 0.001
# 3=0.58715596330275233, ->0.633 0.001
# 4=0.6009174311926605, -> 0.628 0.001
# 5=0.610(alpha = 1e-2 0.6239)(1e-1 0.617)
# 6=0.6223241590214067

0.6223241590214067

In [404]:
text_clf = Pipeline([('vect', CountVectorizer(ngram_range=(1,4))),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=20,max_iter=5, tol=None)),])
text_clf.fit(a_train, b_train)  
predicted = text_clf.predict(a_test)
np.mean(predicted == b_test)  
# 0.65749235474006118  ngram_range=(1,4)

0.65749235474006118

In [405]:
# parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
# #               'vet__lowercase':(True,False),
#               'clf__alpha': (1e-2, 1e-3),
# }
# gs_clf = GridSearchCV(text_clf, parameters, cv=5, iid=False, n_jobs=-1)
# gs_clf = gs_clf.fit(df_group.textual_comb,df_group.label)
# for param_name in sorted(parameters.keys()):
#     print("%s: %r" % (param_name, gs_clf.best_params_[param_name]))
print(gs_clf.best_score_)

In [331]:
#https://stackoverflow.com/questions/26569478/performing-grid-search-on-sklearn-naive-bayes-multinomialnb-on-multi-core-machin
    

In [377]:
Y = df_group.label
X = df_group.textual_comb
from sklearn import linear_model
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import VotingClassifier


unigram_log_pipe = Pipeline([
    ('cv', CountVectorizer()),
    ('logreg', linear_model.LogisticRegression())
])

ngram_pipe = Pipeline([
    ('cv', CountVectorizer(ngram_range=(1, 2))),
    ('mnb', MultinomialNB())
])

tfidf_pipe = Pipeline([
    ('tfidf', TfidfVectorizer(use_idf = False
                              )),
    ('mnb', MultinomialNB())
])

classifiers = [
#     ("ngram", ngram_pipe),
    ("unigram", unigram_log_pipe),
    ("tfidf", tfidf_pipe),
]

mixed_pipe = Pipeline([
    ("voting", VotingClassifier(classifiers, voting="soft"))
])

In [378]:
def combinations_on_off(num_classifiers):
    return [[int(x) for x in list("{0:0b}".format(i).zfill(num_classifiers))]
            for i in range(1, 2 ** num_classifiers)]

param_grid = dict(
    voting__weights=combinations_on_off(len(classifiers))
)

grid_search = GridSearchCV(mixed_pipe, param_grid=param_grid, n_jobs=-1, verbose=10, scoring="neg_log_loss")

grid_search.fit(X, Y)

cv_results = grid_search.cv_results_

for mean_score, params in zip(cv_results["mean_test_score"], cv_results["params"]):
    print(params, mean_score)

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(param_grid.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Fitting 3 folds for each of 3 candidates, totalling 9 fits
[CV] voting__weights=[0, 1] ..........................................
[CV] voting__weights=[0, 1] ..........................................
[CV] voting__weights=[0, 1] ..........................................
[CV] voting__weights=[1, 0] ..........................................
[CV]  voting__weights=[0, 1], score=-1.6297249128372548, total=   5.3s
[CV] voting__weights=[1, 0] ..........................................
[CV]  voting__weights=[0, 1], score=-1.3037999134087996, total=   5.4s
[CV] voting__weights=[1, 0] ..........................................
[CV]  voting__weights=[0, 1], score=-1.4404642106298107, total=   6.1s
[CV] voting__weights=[1, 1] ..........................................
[CV] . voting__weights=[1, 0], score=-2.321808167499604, total=   5.9s
[CV] voting__weights=[1, 1] ..........................................


[Parallel(n_jobs=-1)]: Done   3 out of   9 | elapsed:    7.2s remaining:   14.5s
[Parallel(n_jobs=-1)]: Done   4 out of   9 | elapsed:    7.4s remaining:    9.2s


[CV] . voting__weights=[1, 0], score=-2.412997038792653, total=   1.2s
[CV] voting__weights=[1, 1] ..........................................


[Parallel(n_jobs=-1)]: Done   5 out of   9 | elapsed:    8.2s remaining:    6.5s


[CV] . voting__weights=[1, 0], score=-2.107421822351222, total=   1.3s


...

[CV] . voting__weights=[1, 1], score=-1.484673384288052, total=   1.4s


[Parallel(n_jobs=-1)]: Done   7 out of   9 | elapsed:    9.4s remaining:    2.7s


[CV]  voting__weights=[1, 1], score=-1.2740355793169027, total=   1.5s
[CV] . voting__weights=[1, 1], score=-1.248309040277653, total=   1.4s


[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   10.0s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   9 out of   9 | elapsed:   10.0s finished


{'voting__weights': [0, 1]} -1.45799634563
{'voting__weights': [1, 0]} -2.28074234288
{'voting__weights': [1, 1]} -1.33567266796
Best score: -1.336
Best parameters set:
	voting__weights: [1, 1]


In [376]:
# pipeline = Pipeline([('vec', CountVectorizer(encoding='cp874',  token_pattern=None)),
#                      ('tfidf', TfidfTransformer()), 
#                      ('clf',MultinomialNB())])
# parameters = {  
# 'vec__max_df': (0.5, 0.625, 0.75, 0.875, 1.0),  
# 'vec__max_features': (None, 5000, 10000, 20000),  
# 'vec__min_df': (1, 5, 10, 20, 50),  
# 'tfidf__use_idf': (True, False),  
# 'tfidf__sublinear_tf': (True, False),  
# 'vec__binary': (True, False),  
# 'tfidf__norm': ('l1', 'l2'),  
# 'clf__alpha': (1, 0.1, 0.01, 0.001, 0.0001, 0.00001)  
# }  
# grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=2)  
# grid_search.fit(df_group.textual_comb,df_group.label)  
# print("Best score: {0}".format(grid_search.best_score_))  
# print("Best parameters set:")  
# best_parameters = grid_search.best_estimator_.get_params()  
# for param_name in sorted(list(parameters.keys())):  
#     print("\t{0}: {1}".format(param_name, best_parameters[param_name]))
# https://stackoverflow.com/questions/26569478/performing-grid-search-on-sklearn-naive-bayes-multinomialnb-on-multi-core-machin